In [79]:
dl_num = 'DL-0420110149646'
dob = '09/02/1976'

In [67]:
parivahan_url = 'https://parivahan.gov.in/rcdlstatus/?pur_cd=101'
html = requests.get(parivahan_url).content

In [138]:
soup = BeautifulSoup(html, 'lxml')

In [13]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from pprint import pprint

# initialize an HTTP session
session = HTMLSession()


def get_all_forms(url):
    """Returns all form tags found on a web page's `url` """
    # GET request
    res = session.get(url)
    # for javascript driven website
    #res.html.render()
    soup = BeautifulSoup(res.html.html, "html.parser")
    return soup.find_all("form")


def get_form_details(form):
    """Returns the HTML details of a form,
    including action, method and list of form controls (inputs, etc)"""
    details = {}
    # get the form action (requested URL)
    action = form.attrs.get("action").lower()
    # get the form method (POST, GET, DELETE, etc)
    # if not specified, GET is the default in HTML
    method = form.attrs.get("method", "get").lower()
    # get all form inputs
    inputs = []
    for input_tag in form.find_all("input"):
        # get type of input form control
        input_type = input_tag.attrs.get("type", "text")
        # get name attribute
        input_name = input_tag.attrs.get("name")
        # get the default value of that input tag
        input_value =input_tag.attrs.get("value", "")
        # add everything to that list
        inputs.append({"type": input_type, "name": input_name, "value": input_value})
    # put everything to the resulting dictionary
    details["action"] = action
    details["method"] = method
    details["inputs"] = inputs
    return details


if __name__ == "__main__":
    url = input('Enter url: ')
    
    # get all form tags
    forms = get_all_forms(url)
    # iteratte over forms
    for i, form in enumerate(forms, start=1):
        form_details = get_form_details(form)
        print("="*50, f"form #{i}", "="*50)
        pprint(form_details)

Enter url: https://parivahan.gov.in/rcdlstatus/?pur_cd=101
================================================== form #1 ==================================================
{'action': '/rcdlstatus/vahan/rcdlhome.xhtml;jsessionid=4fae6e26ca591b35ac335f7a19c7e937',
 'inputs': [{'name': 'form_rcdl', 'type': 'hidden', 'value': 'form_rcdl'},
            {'name': 'form_rcdl:tf_dlNO', 'type': 'text', 'value': ''},
            {'name': 'form_rcdl:tf_dob_input', 'type': 'text', 'value': ''},
            {'name': 'form_rcdl:j_idt34:CaptchaID',
             'type': 'text',
             'value': ''},
            {'name': 'javax.faces.ViewState',
             'type': 'hidden',
             'value': 'JtmAMZsyOniymYsb7f7RUxooCxfOnNtMLsRNIVMd3eS8WI/Ew5t9qjp5bWHZKmNw5BqnSha/u9SF+APAOW2sZmWO+viiqA7ag/pgin8C69ZffjQ/Ke0N/G35ejbd7ClAAI3Gviwir5PXLcy//srT2al4N4fsw4Cw8O/3ZB5NaCDY7F7MvmqiyVzD0722ajbUE6jqixaHy3dpHgB5AJpmZNNfOr8wMN6k9DrBY6FFsgXe7hKBmSE7cCkqpwnQPr/7OjtvFv5aZgs3f6NbO3hOcXNqEG/53pC9ZTtGVkvPiiLB6hcBqBSN

In [14]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from pprint import pprint
from urllib.parse import urljoin
import webbrowser


url = input("Enter url here: ")
# get the first form
first_form = get_all_forms(url)[0]
# extract all form details
form_details = get_form_details(first_form)
pprint(form_details)
# the data body we want to submit
data = {}
for input_tag in form_details["inputs"]:
    if input_tag["type"] == "hidden":
        # if it's hidden, use the default value
        data[input_tag["name"]] = input_tag["value"]
    elif input_tag["type"] != "submit":
        # all others except submit, prompt the user to set it
        value = input(f"Enter the value of the field '{input_tag['name']}' (type: {input_tag['type']}): ")
        data[input_tag["name"]] = value

# join the url with the action (form request URL)
url = urljoin(url, form_details["action"])

if form_details["method"] == "post":
    res = session.post(url, data=data)
elif form_details["method"] == "get":
    res = session.get(url, params=data)

# the below code is only for replacing relative URLs to absolute ones
soup = BeautifulSoup(res.content, "html.parser")
for link in soup.find_all("link"):
    try:
        link.attrs["href"] = urljoin(url, link.attrs["href"])
    except:
        pass
for script in soup.find_all("script"):
    try:
        script.attrs["src"] = urljoin(url, script.attrs["src"])
    except:
        pass
for img in soup.find_all("img"):
    try:
        img.attrs["src"] = urljoin(url, img.attrs["src"])
    except:
        pass
for a in soup.find_all("a"):
    try:
        a.attrs["href"] = urljoin(url, a.attrs["href"])
    except:
        pass

# write the page content to a file
open("page.html", "w").write(str(soup))
# open the page on the default browser
webbrowser.open("page.html")

Enter url here: https://parivahan.gov.in/rcdlstatus/?pur_cd=101
{'action': '/rcdlstatus/vahan/rcdlhome.xhtml',
 'inputs': [{'name': 'form_rcdl', 'type': 'hidden', 'value': 'form_rcdl'},
            {'name': 'form_rcdl:tf_dlNO', 'type': 'text', 'value': ''},
            {'name': 'form_rcdl:tf_dob_input', 'type': 'text', 'value': ''},
            {'name': 'form_rcdl:j_idt34:CaptchaID',
             'type': 'text',
             'value': ''},
            {'name': 'javax.faces.ViewState',
             'type': 'hidden',
             'value': 'boLLyhfyOHk9drW0AOTX/BibRqYzlWaddPxeQx2ERM1vTVzz/ypJnUZSC94IGeJccxlqvSBMmB3dZDuGXbxFo2ZapNDBQLGmZQEiSaLy/oiZPzX7EawbeOzeguaOJAZ62Hf3mcxfT6S5lrhaH6c60Ar22VAC/Gf7TJCUnjef89L2NLq+AbBAlwgzvysxUz5wcnEQJNexAu8HMH6twZGhtLv9Brbosnx6IWI8fS2GWbOfJWGlAc5mPU9218PcrXwLx2ftgn2khrUar4NgiSvH+kJQ/0mP7vGejRS/aiY3L6A7jrOdSJkp0HHfIbsTcxmDccnBQX/gJ+dnuLukXADPCezWPIeDEnneAHNPHg51RNAMEaSqS8tpGoNPo1OgVEQIUbfMWlIZITor1tuHG7mnTUtvly04o95PlUfCcBr0WZ9v64v7H+miSfN6CI6X2BGKqHaSbLSjM

True